# 1. Understand the Project Structure
The original project uses symptom descriptions (text) → LSTM → disease prediction.

Since you want to start with a standard neural network, we CANNOT feed raw text directly.
Instead, we convert text into numerical features using something like:

- Bag-of-Words (BoW)

- TF-IDF

- Word embeddings averaged (optional)

For beginners, TF-IDF is the easiest.

# Step 2 - Load the dataset

In [67]:
# Verify NumPy installation and import
import sys
print(f"Python: {sys.version}")
print(f"Python executable: {sys.executable}")

try:
    import numpy as np
    print(f"NumPy version: {np.__version__}")
    print(f"NumPy location: {np.__file__}")
except Exception as e:
    print(f"NumPy import error: {e}")

Python: 3.10.19 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 16:41:31) [MSC v.1929 64 bit (AMD64)]
Python executable: d:\2024\anaconda\envs\cuda_env\python.exe
NumPy version: 1.26.4
NumPy location: d:\2024\anaconda\envs\cuda_env\lib\site-packages\numpy\__init__.py


In [68]:
import pandas as pd
dataset = pd.read_csv("../Dataset/medical_data.csv")

In [69]:
dataset.head()

,Patient_Problem,Disease,Prescription
0,"Constant fatigue and muscle weakness, struggli...",Chronic Fatigue Syndrome,"Cognitive behavioral therapy, graded exercise ..."
1,"Frequent severe migraines, sensitivity to ligh...",Migraine with Aura,"Prescription triptans, avoid triggers like bri..."
2,"Sudden weight gain and feeling cold, especiall...",Hypothyroidism,Levothyroxine to regulate thyroid hormone levels.
3,"High fever, sore throat, and swollen lymph nod...",Mononucleosis,"Rest and hydration, ibuprofen for pain."
4,"Excessive thirst and frequent urination, dry m...",Diabetes Mellitus,Insulin therapy and lifestyle changes.


In [70]:
dataset.tail(5)

,Patient_Problem,Disease,Prescription
402,Noticeable thinning of the hair on the top of ...,Androgenetic Alopecia,"Minoxidil, finasteride for males."
403,"Greenish discharge from the eyes, accompanied ...",Conjunctivitis,Antibiotic or antihistamine eye drops.
404,"Experiencing confusion, difficulty speaking, a...",Stroke,"Immediate medical attention, clot-busting drugs."
405,Constant feeling of fullness in the ears and h...,Eustachian Tube Dysfunction,"Nasal steroids, autoinflation exercises."
406,"Blood in stool, along with a change in bowel m...",Colorectal Cancer,"Colonoscopy, potentially surgery, chemotherapy."


In [71]:
# Combine Patient_Problem and Prescription into one text field
texts = dataset['Patient_Problem']
labels = dataset['Disease']
print(texts.head())
print("------------")
print(labels.head())

0    Constant fatigue and muscle weakness, struggli...
1    Frequent severe migraines, sensitivity to ligh...
2    Sudden weight gain and feeling cold, especiall...
3    High fever, sore throat, and swollen lymph nod...
4    Excessive thirst and frequent urination, dry m...
Name: Patient_Problem, dtype: object
------------
0    Chronic Fatigue Syndrome
1          Migraine with Aura
2              Hypothyroidism
3               Mononucleosis
4           Diabetes Mellitus
Name: Disease, dtype: object


In [72]:
numberofclass = labels.value_counts()

In [73]:
print(numberofclass.shape)

(178,)


# Step 3 - Convert text -> Nummerical vectors (TF-IDF)

In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

# Convert text to TF-IDF vectors
vectorizer = TfidfVectorizer(max_features=2000)
X = vectorizer.fit_transform(texts).toarray()

# Encode disease labels into numbers
encoder = LabelEncoder()
y = encoder.fit_transform(labels)

# Step 4 - Train/Test split

In [75]:
from sklearn.model_selection import train_test_split
import torch

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

# Convert to pytorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [76]:
print(X_train.shape[0])

325


# Step 5 - Build a standard neural network

In [77]:
import torch.nn as nn 
import torch.nn.functional as F

class MLP(nn.Module): # MLP = multilayer perceptron
    def __init__(self, input_dim, num_classes):
        super(MLP,self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, num_classes)

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)

        return X

**Initialize the model**

In [78]:
input_dim = X_train.shape[1]
num_classes = len(encoder.classes_)
model = MLP(input_dim, num_classes)

In [79]:
print(input_dim)
print(num_classes)
print(model)

616
178
MLP(
  (fc1): Linear(in_features=616, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=178, bias=True)
)


# Step 6 - Loss Function and optimizer

In [80]:
criteration = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Step 7 - Training loop

In [84]:
epochs = 1000

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad() # before we compute new gradients, we must clear old gradients

    outputs = model(X_train) # it does: X_train → fc1 → relu → fc2 → relu → output layer
    loss = criteration(outputs, y_train)

    loss.backward() # pytorch computes the gradient of loss with reepect to every weight in the network
    optimizer.step()

    if (epoch + 1) % 2 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [2/1000], Loss: 0.0043
Epoch [4/1000], Loss: 0.0043
Epoch [6/1000], Loss: 0.0043
Epoch [8/1000], Loss: 0.0043
Epoch [10/1000], Loss: 0.0043
Epoch [12/1000], Loss: 0.0043
Epoch [14/1000], Loss: 0.0043
Epoch [16/1000], Loss: 0.0043
Epoch [18/1000], Loss: 0.0043
Epoch [20/1000], Loss: 0.0043
Epoch [22/1000], Loss: 0.0043
Epoch [24/1000], Loss: 0.0043
Epoch [26/1000], Loss: 0.0043
Epoch [28/1000], Loss: 0.0043
Epoch [30/1000], Loss: 0.0043
Epoch [32/1000], Loss: 0.0043
Epoch [34/1000], Loss: 0.0043
Epoch [36/1000], Loss: 0.0043
Epoch [38/1000], Loss: 0.0043
Epoch [40/1000], Loss: 0.0043
Epoch [42/1000], Loss: 0.0043
Epoch [44/1000], Loss: 0.0043
Epoch [46/1000], Loss: 0.0043
Epoch [48/1000], Loss: 0.0043
Epoch [50/1000], Loss: 0.0043
Epoch [52/1000], Loss: 0.0043
Epoch [54/1000], Loss: 0.0043
Epoch [56/1000], Loss: 0.0043
Epoch [58/1000], Loss: 0.0043
Epoch [60/1000], Loss: 0.0043
Epoch [62/1000], Loss: 0.0043
Epoch [64/1000], Loss: 0.0043
Epoch [38/1000], Loss: 0.0043
Epoch [40/1000

# Step 8 - Evaluate accuracy

In [90]:
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs,1)

accuracy = (predicted == y_test).float().mean()
print("Accuracy: ", accuracy.item())

Accuracy:  0.39024388790130615


# Step 9 - Predict on new sysptoms

In [91]:
def predict(symptoms_text):
    model.eval()
    x = vectorizer.transform([symptoms_text]).toarray()
    x = torch.tensor(x, dtype=torch.float32)
    
    with torch.no_grad():
        output = model(x)
        _, pred = torch.max(output, 1)
        
    disease = encoder.inverse_transform(pred.numpy())[0]
    return disease

print(predict("fever and dry cough"))


Pneumonia
